# Preprocessing

In [1]:
from zakupkiClient import *
import nltk

import pandas as pd
import os
import json

os.getcwd()

'/Users/macbook/PycharmProjects/zakupki/notebooks'

In [2]:
ag_name="закупка+автомобилей"
categ="29.10.2 "

In [3]:
#nltk.download()
# import pdb; pdb.set_trace()

In [4]:
stub=Stub(query=ag_name, numFz="223", target=categ)
parser=Parser223(stub)
check_website_up(stub)

loading  http://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber=31807061497


True

In [5]:
parser.get_stub()

# Get Content

Ask search engine to look up the query then go through search pages and save pages

In [ ]:
parser.search_save(offset=1, p_limit=100)

Go through saved search pages and get purchases data then dump it

In [ ]:
parser.parse_save_search_entries(offset=11, p_limit=20)

Openning dir ./data/закупка+автомобилей/search/page_%s.html
Parsed 31806963471 page
Parsed 31806996130 page
Parsed 31806868621 page
Parsed 31806996438 page
Parsed 31806969653 page
Parsed 31806993558 page
Parsed 31806650610 page
Parsed 31806952719 page
Parsed 31806709151 page


In [7]:
purchases_json=load_JSON_data(stub=stub, filename="db.json")
#10 puchases per 1 search page

In [8]:
len(purchases_json)

100

In [ ]:
purchases_df=pd.DataFrame.from_records(purchases,index=range(1,len(purchases)+1))

pd.set_option('display.max_colwidth', -1)

purchases_df

When all purchases' data is stored go through purchases and create lots database,  dump it

In [6]:
lots=parser.create_save_lots()
len(lots)

104

When all lots data is stored, go through lots and update their vendors property

In [ ]:
lots=parser.get_vendors_save_lots(lots=lots)

In [ ]:
len(lots)

# Load Content

In [ ]:
lots_json=load_JSON_data(stub=stub, filename="lots.json")

In [ ]:
df=pd.DataFrame.from_records(lots,index=range(1,len(lots)+1))

pd.set_option('display.max_colwidth', -1)

df

# Clustering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
corpus = df["name"].tolist()
corpus=[clear_text(l) for l in corpus]
corpus

In [ ]:
vectorizer = CountVectorizer()
print( vectorizer.fit_transform(corpus).todense() )
print( vectorizer.vocabulary_ )

In [ ]:
# load nltk's russian stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('russian')

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [ ]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
import re
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in corpus:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer( max_features=200000,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(corpus) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [ ]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [ ]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print ("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print (' %s' % terms[ind])
    print()

In [ ]:
page=load_page(get_purchase_tab(31807203064,"protocols"),ses)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, features="lxml")

In [ ]:
toolTipMenuDiv=soup.find("div",{"class":"toolTipMenu"})

In [ ]:
command=toolTipMenuDiv.find("li").get('onclick')
command

In [ ]:
link=command[command.find("\'")+1:]
link

In [ ]:
link=link[:link.find("\'")]
link

In [ ]:
fulllink="http://zakupki.gov.ru%s"
fulllink%link

In [ ]:
page=load_page("http://zakupki.gov.ru/223/purchase/public/print-form/show.html?pfid=31640023",ses)
page

In [ ]:
soup = BeautifulSoup(page, features="lxml")
tabs1=soup.find("div",{"id":"tabs-1"})

In [ ]:
trs=tabs1.find_all("tr")


In [ ]:
grades={}
i=0
k=-1
for tr in trs:
    td_text = [td.text.strip() for td in tr.find_all("td")]
    grades[i] = td_text
    if "Выбор победителя:" in td_text:
        k = i
        break
    i += 1
if "Заявок нет" in trs[k]:
    print("NA")

In [ ]:
tds=trs[k].find_all("td")
tds


In [ ]:
tds

In [ ]:
if "Заявок нет" in tds:
    print("NA")